In [ ]:
import geopandas
import scipy.io
import pandas as pd
import numpy as np
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

In [ ]:
# get geopandas representation of final_biomass_table
df = epro_2019.final_biomass_table

krig_mesh = epro_2019.get_kriging_mesh()

# apply transformations to data using python functions
trans_df = krig_mesh.apply_longitude_transformation(df)

D_x = trans_df['Longitude'].max() - trans_df['Longitude'].min()
D_y = trans_df['Latitude'].max() - trans_df['Latitude'].min()
x_data, y_data = krig_mesh.apply_distance_transformation(trans_df, D_x, D_y)

field_data = df['nwgt_total'].values.flatten()

In [ ]:
# apply transformations to data using python functions
trans_dfm = krig_mesh.apply_longitude_transformation(krig_mesh.mesh_gdf, 
                                                    gdf_lon_name='Longitude of centroid', 
                                                    gdf_lat_name='Latitude of centroid')

x_mesh, y_mesh = krig_mesh.apply_distance_transformation(trans_dfm, D_x, D_y,
                                                           gdf_lon_name='Longitude of centroid',
                                                           gdf_lat_name='Latitude of centroid')

In [ ]:
from EchoPro.semivariogram import SemiVariogram as SV

In [ ]:
%%time 
krig = epro_2019.get_kriging()

k_max = 10
k_min = 3
R = 0.0226287
ratio = 0.001
s_v_params = {'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
              'exp_pow': 1.5, 'ls_hole_eff': 0.0}
s_v_model = SV.generalized_exp_bessel

ep_arr, eps_arr, vp_arr = krig.run_kriging(x_mesh, x_data, 
                                           y_mesh, y_data, field_data, 
                                           k_max, k_min, R, ratio, 
                                           s_v_params, s_v_model)

In [ ]:
# get kriging output data from Matlab version of EchoPro
# krig_output_mat = scipy.io.loadmat('../2019_consolidated_files/kriging_output_vals.mat')
krig_output_mat = scipy.io.loadmat('../2019_consolidated_files/kriging_output_vals_m2_unity_chng_bug_fix.mat')
ep_matlab = krig_output_mat['Ep'].flatten()
eps_matlab = krig_output_mat['Eps'].flatten()
vp_matlab = krig_output_mat['Vp'].flatten()

print(np.allclose(ep_arr, ep_matlab))
print(np.allclose(eps_arr, eps_matlab, equal_nan=True))
print(np.allclose(vp_arr, vp_matlab))

In [ ]:
# k_mesh_x = krig_mesh.mesh_gdf['Latitude of centroid'].values.flatten()
# k_mesh_y = krig_mesh.mesh_gdf['Longitude of centroid'].values.flatten()

# krig.plot_kriging_results(k_mesh_x, k_mesh_y, vp_arr)